##**Gerrymandering-Environment**

    INITIAL STATE (provided externally via reset(options=...)):
        - 'district_map'
        - 'social_graph'
        - 'opinions'     

    ACTION:
        - new district assignment for each voter

    OBSERVATION (returned by reset/step):
        {
          'district_map'   : (num_voters,)
          'representatives': (num_districts,)  # voter indices; -1 if empty district
          'social_graph'   : (num_voters, num_voters)  # AUGMENTED: base social + rep->voter edges used for the step
          'opinions'       : (num_voters, 2)
          'opinion_graph'  : (num_voters, num_voters)  # similarity kernel derived from opinion distances
        }

    KEY LOGIC:
      - Representatives: for each district, pick the member that minimizes the sum of L2 distances to members in that district (discrete 1-median).
      - Opinion dynamics: DRF (assimilation/neutral/backfire) with weighted neighbor influence.
      - Reward: reduction in total distance to reference opinion c*

    Notes:
      - Opinion weight = 1
      - Opinion dimension is fixed at 2.
      - We accept any districting action

In [1]:
!pip install torch_geometric

In [2]:
import numpy as np
import torch
from torch_geometric.data import Data
import gymnasium as gym
from gymnasium import spaces
import gerry_environment_12

In [3]:
import copy
import numpy as np

def simulated_annealing_baseline(env, num_iterations=500,
                                 initial_beta=0.1, cooling_rate=1.01, rng=None):
    if rng is None:
        rng = np.random.default_rng()

    # initialize
    current_assignment = np.eye(env.num_districts)[
        rng.integers(0, env.num_districts, size=env.num_voters)
    ]

    # evaluate initial plan
    saved_state = copy.deepcopy(env.FrankenData)
    _, current_reward, _, _, _ = env.step(current_assignment)

    env.FrankenData = saved_state  # revert so we don’t advance time
    best_assignment = current_assignment.copy()
    best_reward = current_reward

    beta = initial_beta

    for _ in range(num_iterations):
        # propose
        proposal_assignment = current_assignment.copy()
        voter = rng.integers(0, env.num_voters)
        new_d = rng.integers(0, env.num_districts)
        proposal_assignment[voter, :] = 0
        proposal_assignment[voter, new_d] = 1.0

        # evaluate
        saved_state = copy.deepcopy(env.FrankenData)
        _, proposal_reward, _, _, _ = env.step(proposal_assignment)
        env.FrankenData = saved_state  # revert to current accepted state

        # acceptance
        if (proposal_reward > current_reward or
            rng.random() < np.exp(beta * (proposal_reward - current_reward))):
            current_assignment = proposal_assignment
            current_reward = proposal_reward
            if current_reward > best_reward:
                best_assignment = current_assignment.copy()
                best_reward = current_reward

        # cool
        beta *= cooling_rate

    return best_assignment, best_reward


In [4]:
import numpy as np
import torch
from torch_geometric.data import Data

num_voters = 6
num_districts = 2
opinion_dim = 2
orig_edge_num = 8

# 1. Opinions: 2D (num_voters, opinion_dim)
opinions = np.random.randn(num_voters, opinion_dim).astype(np.float32)

# 2. Positions: 2D coordinates
positions = np.random.rand(num_voters, 2).astype(np.float32)

# 3. District labels: assign evenly
dist_labels = np.array([i % num_districts for i in range(num_voters)], dtype=np.int32)

# 4. Representatives: first voter in each district
reps = np.array([np.where(dist_labels == d)[0][0] for d in range(num_districts)], dtype=np.int32)

# 5. Social edges: random edges, avoid self-loops
src = np.random.randint(0, num_voters, size=orig_edge_num*2)
dst = np.random.randint(0, num_voters, size=orig_edge_num*2)
mask = src != dst
src, dst = src[mask][:orig_edge_num], dst[mask][:orig_edge_num]  # keep exact number
social_edge_index = np.vstack([src, dst])

# 6. Edge attributes: random positive weights
edge_attr = np.random.rand(orig_edge_num).astype(np.float32)

# 7. Assignment: (district, voter) pairs
assignment = np.vstack([dist_labels, np.arange(num_voters)])

# 8. Create FrankenData
initial_data = gerry_environment_12.FrankenData(
    so_edge=social_edge_index,
    assignment=assignment,
    orig_edge_num=orig_edge_num,
    opinion=opinions,
    pos=positions,
    reps=reps,
    dist_label=dist_labels,
    edge_attr=edge_attr
)

print("FrankenData initialized:")
print("Opinions shape:", initial_data.opinion.shape)        # (num_voters, opinion_dim)
print("Positions shape:", initial_data.pos.shape)           # (num_voters, 2)
print("Dist labels:", initial_data.dist_label)              # (num_voters,)
print("Reps:", initial_data.reps)                           # (num_districts,)
print("Social edges shape:", initial_data.so_edge.shape)    # (2, orig_edge_num)
print("Edge attributes shape:", initial_data.edge_attr.shape)  # (orig_edge_num,)
print("Assignment shape:", initial_data.dist_edge.shape)    # (2, num_voters)


FrankenData initialized:
Opinions shape: torch.Size([6, 2])
Positions shape: torch.Size([6, 2])
Dist labels: tensor([0., 1., 0., 1., 0., 1.])
Reps: tensor([0, 1])
Social edges shape: torch.Size([2, 8])
Edge attributes shape: torch.Size([8])
Assignment shape: torch.Size([2, 6])


In [6]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Create a small environment
env = gerry_environment_12.FrankenmanderingEnv(
    num_voters=num_voters,
    num_districts=num_districts,
    opinion_dim=opinion_dim,
    FrankenData=initial_data
)


# 2. Run baseline
best_assignment, best_reward = simulated_annealing_baseline(
    env,
    num_iterations=200,
    initial_beta=0.05,
    cooling_rate=1.02
)
print("Best reward found:", best_reward)

# 3. Inspect or visualize
y = best_assignment.argmax(axis=1)
print("District counts:", np.bincount(y + 1))  # +1 to avoid -1

# Optional: run multiple times to see cooling behaviour
rewards = []
beta = 0.05
curr_assign, curr_reward = best_assignment, best_reward
for i in range(200):
    rewards.append(curr_reward)
    beta *= 1.02


Best reward found: 2.8473741531372063
District counts: [0 0 6]
